In [1]:
import requests

# GET Request
get_url = "http://localhost:9001/v1/models/forecast_currency_exchange_rate"
response_get = requests.get(get_url)
print(response_get.json())

# POST Request
post_url = "http://localhost:9001/v1/models/forecast_currency_exchange_rate/versions/1:predict"
headers = {"Content-Type": "application/json"}


{'model_version_status': [{'version': '10', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '9', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '8', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '7', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '6', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '5', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '4', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '3', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '2', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}, {'version': '1', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [8]:
from datetime import datetime
import pandas as pd

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')

series = pd.read_csv('./currency_data_10_years.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
series.head()

columns = series.columns.tolist()
future_forecasts=[]

into_future = 10
for column in columns:
    future_forecast=[]
    input_series = series[column].tolist()[-30:] # window_size = 30
    print(f"{column}/IDR {into_future} days forecast")
    for i in range(into_future):
        data = {
            "instances": [
                input_series
            ]
        }
        response_post = requests.post(post_url, headers=headers, json=data)
        prediction = response_post.json()["predictions"][0][0]
        future_forecast.append(prediction)
        print(prediction)
        
        input_series.append(prediction)
        input_series = input_series[1:]
    future_forecasts.append(future_forecast)

        

<ipython-input-8-400178ccf1fc>:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  series = pd.read_csv('./currency_data_10_years.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)


EUR/IDR 10 days forecast
16259.0928
16274.2275
16294.1465
16262.4961
16320.0449
16352.1973
16359.0967
16310.8975
16318.8564
16344.5596
USD/IDR 10 days forecast
14749.4199
14772.4238
14781.0957
14741.5811
14767.0068
14787.3877
14818.3887
14810.6211
14805.7
14804.5264
JPY/IDR 10 days forecast
108.719048
108.794777
108.780731
108.737778
108.767883
109.072578
109.618683
109.630676
109.756653
109.843323
GBP/IDR 10 days forecast
18451.3496
18459.0605
18482.6328
18455.4121
18498.0566
18508.6348
18526.2
18492.0449
18492.7578
18534.7148
SGD/IDR 10 days forecast
11048.4053
11049.7256
11056.9092
11036.9502
11057.2891
11071.6895
11087.3125
11075.9502
11070.7451
11076.7217
AUD/IDR 10 days forecast
9799.92676
9801.23926
9793.53223
9793.72656
9821.14258
9838.37305
9834.14453
9822.8252
9801.55859
9809.86328
CNY/IDR 10 days forecast
2135.43726
2136.75171
2138.00903
2133.69165
2137.86182
2140.86719
2143.95532
2141.47827
2139.75
2139.85254
CAD/IDR 10 days forecast
10889.6445
10877.4629
10868.7207
10857.5

In [9]:
print(future_forecasts[0])

[16259.0928, 16274.2275, 16294.1465, 16262.4961, 16320.0449, 16352.1973, 16359.0967, 16310.8975, 16318.8564, 16344.5596]
